In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import folium
from folium.plugins import MarkerCluster
import time
import urllib.request
import ipyspin

In [ ]:
%%HTML

<style>
.jp-Cell {
    width: 800px;
    margin: 0 auto;
}
:root {
  --h1-color: "#242138";
}
h1 { text-align: center; color: var(--h1-color); }
<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@picocss/pico@1/css/pico.min.css">
</style>


In [ ]:
DB_NAME = "/Users/adrienhenry/Documents/house_pricing/data/commune.db"
SLIDER_STEPS = 100


In [ ]:
def get_variables():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    variable_dict = {}
    for row in cursor.execute("PRAGMA table_info(commune);").fetchall():
        variable_dict[row[1]] = row[2]
    conn.close()
    return variable_dict


def get_variable_bounds(variable):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    bounds = cursor.execute(
        f"SELECT min({variable}), max({variable}) FROM commune;"
    ).fetchall()[0]
    conn.close()
    return bounds


def genereate_city_marker(row, cityCluster):
    marker = folium.Marker(
        location=[row["latitude"], row["longitude"]],
        icon=folium.Icon(color="red", icon="glyphicon-home", prefix="glyphicon"),
        popup=folium.Popup(f"<b>{row['name']}</b>", max_width=300),
    )
    marker.add_to(cityCluster)


class Filter_Container:
    def __init__(self):
        self._variables = get_variables()
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        self._data = pd.read_sql_query("SELECT * FROM commune;", conn)
        self._filtered_data = self._data.copy()
        self._filters = {}
        conn.close()
        self.init_selector_widget()

    def get_selectable_keys(self):
        return [
                key
                for key, ktype in self._variables.items()
                if key not in ["code_commune_INSEE", "code_postal"] and ktype != "TEXT"
            ]
     

    def init_selector_widget(self):
        self._label_w = widgets.Label(
            value="Select variables (hold ctrl/CMD to select multiple):"
        )
        self._var_select_w = widgets.SelectMultiple(
            options=self.get_selectable_keys(),
            value=[],
            rows=5,
            disabled=False,
            sytle={"description_width": "initial"},
        )
        self._num_city_w = widgets.VBox(
            [
                widgets.Label(value="Number of cities:"),
                widgets.HBox(
                    [
                        widgets.IntProgress(
                            value=len(self._data),
                            min=0,
                            max=len(self._data),
                            style={"bar_color": "maroon"},
                            orientation="horizontal",
                        ),
                        widgets.Label(value=str(len(self._data))),
                    ]
                ),
            ]
        )
        self._filter_container = widgets.VBox()
        self._map_button = widgets.Button(description="Generate map")
        self._map_button.on_click(lambda x: self.generate_map())
        self._map_path_w = widgets.Text(value="map.html")
        self._map_placeholder = widgets.Output()
        self._selector_box_w = widgets.VBox(
            [
                self._var_select_w,
                self._filter_container,
                self._num_city_w,
                widgets.HBox([self._map_button, self._map_path_w]),
                self._map_placeholder,
            ],
            layout=widgets.Layout(width="80%"),
        )
        self._var_select_w.observe(self._on_variable_select, names="value")


    def _on_variable_select(self, change):
        for key in change["new"]:
            if key not in self._filters.keys():
                self.add_filter(key, self._variables[key])
        for key in change["old"]:
            if key not in change["new"]:
                del self._filters[key]
        self.refresh_widget()

    def update_number_of_cities(self):
        self._num_city_w.children[1].children[0].value = self._filtered_data.shape[0]
        self._num_city_w.children[1].children[1].value = str(self._filtered_data.shape[0])

    def filter_data(self, x):
        self._filtered_data = self._data.copy()
        for key, filter in self._filters.items():
            self._filtered_data = self._filtered_data[
                (self._filtered_data[key] >= filter._range_w.value[0])
                & (self._filtered_data[key] <= filter._range_w.value[1])
            ]
        self.update_number_of_cities()

    def refresh_widget(self):
        self._filter_container.children = [
            filter.get_widget() for filter in self._filters.values()
        ]

    def add_filter(self, key, type):
        self._filters[key] = Range_Selector(key, type, self.filter_data)

    def display_hist(self):
        display(self._hist_w)

    def display_selector(self):
        display(self._selector_box_w)

    def generate_map(self):
        self._map_placeholder.clear_output()
        t0 = time.time()
        with self._map_placeholder:
            display(ipyspin.Spinner())
        m = folium.Map((47, 2), zoom_start=6.4, tiles="cartodb positron")
        cityCluster = MarkerCluster(name="cities").add_to(m)
        self._filtered_data.apply(
            genereate_city_marker, axis=1, cityCluster=cityCluster
        )
        m.save(self._map_path_w.value)
        t1 = time.time()
        self._map_placeholder.clear_output()
        with self._map_placeholder:
            display(widgets.HTML(f"Map generated: {self._map_path_w.value}"))
            if t1 - t0 < 10:
                display(m)


class Range_Selector:
    def __init__(self, key, type, update_fct) -> None:
        self._key = key
        bounds = get_variable_bounds(key)
        widget_fct = (
            widgets.FloatRangeSlider if type == "REAL" else widgets.IntRangeSlider
        )

        self._label_w = widgets.Label(value=f"{key}:")
        self._range_w = widget_fct(
            value=bounds,
            min=bounds[0],
            max=bounds[1],
            disabled=False,
            continuous_update=False,
            orientation="horizontal",
            readout=True,
            layout=widgets.Layout(width="90%"),
        )
        self._range_w.observe(update_fct, names="value")
        self._widget = widgets.VBox([self._label_w, self._range_w])

    def get_selected_range(self):
        return self._min, self._max

    def get_widget(self):
        return self._widget

# Find your city

In [ ]:
container = Filter_Container();

### Select variables (hold `ctrl` / `CMD` to select multiple):

In [ ]:
container.display_selector()